In [1]:
import pandas as pd
import numpy as np

# Preparacion de datos

In [21]:
def get_postulaciones():
    postulaciones = pd.read_csv('data/FiubaHasta15Abril/fiuba_4_postulaciones.csv')
    columns_rename = {'idaviso': 'id_aviso', 'idpostulante': 'id_postulante', 'fechapostulacion': 'fecha_postulacion'}
    postulaciones = postulaciones.rename(columns=columns_rename)
    postulaciones['fecha_postulacion']=pd.to_datetime(postulaciones['fecha_postulacion'])
    return postulaciones

def get_vistas():
    vistas = pd.read_csv('data/FiubaHasta15Abril/fiuba_3_vistas.csv')
    columns_rename = {'idAviso': 'id_aviso', 'idpostulante': 'id_postulante', 'timestamp': 'fecha_vista'}
    vistas = vistas.rename(columns=columns_rename)
    vistas['fecha_vista']=pd.to_datetime(vistas['fecha_vista'])
    
    return vistas

def get_avisos_detalle():
    avisos_detalle = pd.read_csv('data/FiubaHasta15Abril/fiuba_6_avisos_detalle.csv')
    columns_rename = {'idpostulante': 'id_postulante', 'idaviso': 'id_aviso'}
    avisos_detalle = avisos_detalle.rename(columns=columns_rename)
    to_nivel_laboral_nro = {'Senior / Semi-Senior' : 2, 'Junior':1, 'Otro':0,
       'Jefe / Supervisor / Responsable':3,
       'Gerencia / Alta Gerencia / Dirección':4}
    avisos_detalle['nivel_laboral_nro']= avisos_detalle['nivel_laboral'].map(to_nivel_laboral_nro)
    return avisos_detalle

def get_year_of_birth(postulantes_genero_edad):
    return (pd.to_datetime
            (postulantes_genero_edad['fechanacimiento'], errors='coerce', format='%Y-%m-%d')
            .dt.year)

def get_age(yearOfBirth):
    return 2018 - yearOfBirth
    
def get_age_range(yearOfBirth):
    age = get_age(yearOfBirth)
    if(age<25): return 'Entre 18 y 24'
    if(age<30): return 'Entre 25 y 30'
    if(age<35): return 'Entre 30 y 35'
    if(age<40): return 'Entre 35 y 40'
    if(age<45): return 'Entre 40 y 45'
    if(age<50): return 'Entre 45 y 50'
    return 'Mayor de 50'

def get_order_for_age_range():
    return ['Entre 18 y 24', 'Entre 25 y 30','Entre 30 y 35','Entre 35 y 40','Entre 40 y 45','Entre 45 y 50', 'Mayor de 50']

def get_postulantes_nivel_educativo():
    postulantes_nivel_educativo = pd.read_csv('data/FiubaHasta15Abril/fiuba_1_postulantes_educacion.csv')
    postulantes_nivel_educativo.head()
    columns_rename = {'idpostulante': 'id_postulante', 'nombre': 'formacion_postulante', 'estado': 'estado_formacion_postulante'}
    postulantes_nivel_educativo=postulantes_nivel_educativo.rename(columns=columns_rename)
    formacion_to_number={'Secundario' : 10, 'Otro': 20, 'Terciario/Técnico' : 30, 'Universitario' : 40, 'Posgrado' : 50,
    'Master' : 50, 'Doctorado' : 50}
    postulantes_nivel_educativo['formacion_postulante_numero']=postulantes_nivel_educativo['formacion_postulante'].map(formacion_to_number);
    estado_to_number = {'En Curso': 4, 'Abandonado': 0, 'Graduado': 8}
    postulantes_nivel_educativo['estado_formacion_postulante_numero']=postulantes_nivel_educativo['estado_formacion_postulante'].map(estado_to_number)
    postulantes_nivel_educativo['nivel_educativo_postulante_numero'] = postulantes_nivel_educativo['formacion_postulante_numero'] + postulantes_nivel_educativo['estado_formacion_postulante_numero']
    postulantes_nivel_educativo['nivel_educativo_postulante_texto'] = postulantes_nivel_educativo['formacion_postulante'] + ' - ' + postulantes_nivel_educativo['estado_formacion_postulante']
    relevant_columns = ['id_postulante','nivel_educativo_postulante_texto', 'nivel_educativo_postulante_numero']
    postulantes_nivel_educativo = postulantes_nivel_educativo[relevant_columns]
    grouped=postulantes_nivel_educativo.groupby(['id_postulante']).agg({'nivel_educativo_postulante_numero':['max']}) 
    df=grouped.reset_index()
    df.columns = ['id_postulante', 'maximo_nivel_educativo_postulante']
    
    return df

def get_postulantes_genero_edad():
    postulantes_genero_edad = pd.read_csv('data/FiubaHasta15Abril/fiuba_2_postulantes_genero_y_edad.csv')
    postulantes_genero_edad['año_nacimiento_postulante']=get_year_of_birth(postulantes_genero_edad)
    postulantes_genero_edad['edad_postulante']=postulantes_genero_edad['año_nacimiento_postulante'].map(get_age, na_action=None)
    postulantes_genero_edad['rango_edad_postulante']=postulantes_genero_edad['año_nacimiento_postulante'].map(get_age_range, na_action=None)
    columns_rename = {'idpostulante': 'id_postulante', 'fechanacimiento': 'fecha_nacimiento_postulante', 'sexo': 'genero_postulante'}
    postulantes_genero_edad = postulantes_genero_edad.rename(columns=columns_rename)
    postulantes_genero_edad = postulantes_genero_edad[['id_postulante', 'genero_postulante', 'fecha_nacimiento_postulante', 'edad_postulante', 'rango_edad_postulante']]
    postulantes_genero_edad['genero_postulante'] = postulantes_genero_edad['genero_postulante'].map({'FEM': 'Femenino', 'MASC': 'Masculino', 'NO_DECLARA': 'No declara'})
    return postulantes_genero_edad


def get_postulantes_limpios():
    postulantes_genero = get_postulantes_genero_edad()
    postulantes_educacion = get_postulantes_nivel_educativo()
    df=postulantes_educacion.set_index('id_postulante').join(postulantes_genero.set_index('id_postulante'))
    df = df.reset_index()
    
    return df[['id_postulante','edad_postulante', 'genero_postulante', 'maximo_nivel_educativo_postulante']]

def get_detalle_postulaciones():
    postulaciones = get_postulaciones()
    avisos = get_avisos_detalle()
    postulantes = get_postulantes_limpios()
    detalle_postulaciones = pd.merge(postulantes, postulaciones, on='id_postulante', how='inner') 
    detalle_postulaciones = pd.merge(detalle_postulaciones, avisos, on='id_aviso', how='inner')
    return detalle_postulaciones

def get_detalle_vistas():
    vistas = get_vistas()
    avisos = get_avisos_detalle()
    postulantes = get_postulantes_limpios()
    detalle_vistas = pd.merge(postulantes, vistas, on='id_postulante', how='inner') 
    detalle_vistas = pd.merge(detalle_vistas, avisos, on='id_aviso', how='inner')
    return detalle_vistas

def se_postulo(row):
    
    row['postulo']= np.max(row[relevantRankColumns])
    return row

def get_postulaciones_no_aplicadas(postulaciones_aplicadas):
    vistas = get_detalle_vistas()
    vistas['postulo'] = False
    columnas_relevantes = ['edad_postulante', 'genero_postulante', 'maximo_nivel_educativo_postulante', 'nivel_laboral', 'tipo_de_trabajo', 'postulo']
    postulaciones_aplicadas['postulo'] = True
    
    return  pd.merge(postulaciones_aplicadas, vistas, on=['id_aviso', 'id_aviso'], how='outer')

def get_detalle_postulaciones_limpias():
    postulaciones_aplicadas = get_detalle_postulaciones()
    columnas_relevantes = ['edad_postulante', 'genero_postulante', 'maximo_nivel_educativo_postulante', 'nivel_laboral', 'tipo_de_trabajo', 'postulo']
    postulaciones_aplicadas['postulo'] = True
    postulaciones_no_aplicadas = get_detalle_vistas().head(4365045)
    postulaciones_no_aplicadas['postulo'] = False
    postulaciones_aplicadas = postulaciones_aplicadas.append(postulaciones_no_aplicadas)
    return postulaciones_aplicadas[columnas_relevantes]

def get_top_for(df, categorical_key, top_size):
    top_values = df[categorical_key].value_counts().head(top_size).index
    return df[df[categorical_key].isin(top_values)]

In [22]:
postulaciones = get_detalle_postulaciones_limpias()

In [23]:
postulaciones['postulo'].value_counts()

True     4365045
False    4365045
Name: postulo, dtype: int64

# Entrenamiento

In [24]:
from sklearn.model_selection import train_test_split
from sklearn import svm

In [27]:
postulaciones=postulaciones.dropna()
columnas_x = ['edad_postulante', 'genero_postulante', 'maximo_nivel_educativo_postulante', 'nivel_laboral', 'tipo_de_trabajo']
columna_y='postulo'
# Prepare feature and dependent variable along with categorical encoding
X=pd.get_dummies(postulaciones.loc[:, columnas_x])
y=postulaciones.loc[:, 'postulo']

In [ ]:
clf = svm.SVC()
clf.fit(X, y)

In [220]:
clf.predict([[12,48,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0]])

array([ True], dtype=bool)

# Prueba y resultados para la competencia

In [ ]:
tests = pd.read_csv('data/Test/test_final100k.csv')
test.head()